# Example 3: Sector coupling

## Learning objectives
* Model different energy sectors
* Model sector-coupling
* Apply the model to minimize costs / emissions
* Analyze different decarbonization scenarios

## Recap
We used yesterday a trimmed down version of urbs to model the power system expansion. We got familiar with the key model components and its nomenclature.
> If you have any question regarding a piece of code, this is the time to discuss it!

## pyomo ConcreteModel with python input

We will continue working with our mini urbs version and apply it on different use cases (all greenfield expansion planning):
1. OldTown: the demands for electricity and heat are covered by different technologies with no coupling
2. NewTown: some technologies can provide both heat and power
3. FutureTown: electricity is used to cover power and heat demands

### Libraries

In [ ]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Data

In [ ]:
def generate_scenario(objective="cost", local_co2_limit=np.inf, co2_price=10):
    """ This function generates for different scenario assumptions.
    Args:
    - objective: either "cost" (default) or "CO2"
    - local_co2_limit: default is infinite
    - co2_price: default is 10€/t
    """
    # To avoid any confusion, we pick a personalized name for our dictionary
    mydata = dict()

    # Parameters
    mydata["dt"] = 1 # time interval between two consecutive steps, in this case 1 hour
    mydata["objective"] = objective
    mydata["Cost limit"] = np.inf # per year
    mydata["CO2 limit"] = np.inf

    # Sets
    #mydata["timesteps"] will be defined based on the length of the time series
    mydata["support_timeframes"] = [2021]
    mydata["sites"] = ["OldTown", "NewTown", "FutureTown"]
    mydata["commodities"] = ["Elec", "Heat", "Gas", "SolarPV", "SolarThermal", "CO2", "AmbientTemp"]
    mydata["com_type"] = ["Demand", "Stock", "Env", "SupIm"] # (i.e. SupIm, Demand, Stock, Env)
    mydata["process"] = ["Gas CC", "PV plant", 
                         "Gas heating plant", "Solar thermal",
                         "Gas CHP", # CHP: combined heat and power
                         "Heat pump", "Electric resistance"] 
    mydata["cost_type"] = ['Invest', 'Fixed', 'Variable', 'Fuel', 'Environmental']

    # Dictionaries - commodities
    mydata["com_prices"] = {}
    mydata["com_max"] = {}
    for stf in mydata["support_timeframes"]:
        for sit in mydata["sites"]:
            mydata["com_prices"].update({(stf, sit, "Gas", "Stock"): 25.2,
                                         (stf, sit, "SolarPV", "SupIm"): 0,
                                         (stf, sit, "SolarThermal", "SupIm"): 0,
                                         (stf, sit, "AmbientTemp", "SupIm"): 0,
                                         (stf, sit, "Elec", "Demand"): 0,
                                         (stf, sit, "Heat", "Demand"): 0,
                                         (stf, sit, "CO2", "Env"): co2_price})

            mydata["com_max"].update({(stf, sit, "Gas", "Stock"): np.inf,
                                      (stf, sit, "SolarPV", "SupIm"): np.inf,
                                      (stf, sit, "SolarThermal", "SupIm"): np.inf,
                                      (stf, sit, "AmbientTemp", "SupIm"): np.inf,
                                      (stf, sit, "Elec", "Demand"): np.inf,
                                      (stf, sit, "Heat", "Demand"): np.inf,
                                      (stf, sit, "CO2", "Env"): local_co2_limit})

    # Dictionaries - processes
    mydata["pro_capup"] = {
        (2021, "OldTown", "Gas heating plant"): np.inf, # for heat
        (2021, "OldTown", "Solar thermal"): np.inf, # for heat
        (2021, "OldTown", "Gas CC"): np.inf, # for electricity
        (2021, "OldTown", "PV plant"): np.inf, # for electricity
        
        (2021, "NewTown", "Gas heating plant"): np.inf, # for heat
        (2021, "NewTown", "Solar thermal"): np.inf, # for heat
        (2021, "NewTown", "Gas CC"): np.inf, # for electricity
        (2021, "NewTown", "PV plant"): np.inf, # for electricity
        (2021, "NewTown", "Gas CHP"): np.inf, # for electricity and heat
        
        (2021, "FutureTown", "Gas CC"): np.inf, # for electricity
        (2021, "FutureTown", "PV plant"): np.inf, # for electricity
        (2021, "FutureTown", "Heat pump"): np.inf, # for heat from electricity
        (2021, "FutureTown", "Electric resistance"): np.inf, # for heat from electricity
    }
    
    mydata["pro_instcap"] = {}
    mydata["pro_caplo"] = {}
    for k in mydata["pro_capup"].keys():
        mydata["pro_instcap"][k] = 0
        mydata["pro_caplo"][k] = 0

    # initialize with pro_instcap to obtain the same keys
    mydata["pro_invcost"] = mydata["pro_instcap"].copy()
    mydata["pro_fixcost"] = mydata["pro_instcap"].copy()
    mydata["pro_varcost"] = mydata["pro_instcap"].copy()
    mydata["pro_wacc"] = mydata["pro_instcap"].copy()
    mydata["pro_depreciation"] = mydata["pro_instcap"].copy()
    
    for (stf, sit, pro) in mydata["pro_invcost"].keys():
        mydata["pro_wacc"][(stf, sit, pro)] = 0.05 # weigthed average cost of capital (in % of capital cost)
        if pro == "Gas CC":
            mydata["pro_invcost"][(stf, sit, pro)] = 900000
            mydata["pro_fixcost"][(stf, sit, pro)] = 22000
            mydata["pro_varcost"][(stf, sit, pro)] = 4
            mydata["pro_depreciation"][(stf, sit, pro)] = 30
        if pro == "PV plant":
            mydata["pro_invcost"][(stf, sit, pro)] = 700000
            mydata["pro_fixcost"][(stf, sit, pro)] = 22000
            mydata["pro_varcost"][(stf, sit, pro)] = 0
            mydata["pro_depreciation"][(stf, sit, pro)] = 25
        if pro == "Gas heating plant":
            mydata["pro_invcost"][(stf, sit, pro)] = 400000
            mydata["pro_fixcost"][(stf, sit, pro)] = 12000
            mydata["pro_varcost"][(stf, sit, pro)] = 6.7
            mydata["pro_depreciation"][(stf, sit, pro)] = 35
        if pro == "Solar thermal":
            mydata["pro_invcost"][(stf, sit, pro)] = 800000
            mydata["pro_fixcost"][(stf, sit, pro)] = 13600
            mydata["pro_varcost"][(stf, sit, pro)] = 0
            mydata["pro_depreciation"][(stf, sit, pro)] = 25
        if pro == "Gas CHP":
            mydata["pro_invcost"][(stf, sit, pro)] = 1000000
            mydata["pro_fixcost"][(stf, sit, pro)] = 20000
            mydata["pro_varcost"][(stf, sit, pro)] = 0
            mydata["pro_depreciation"][(stf, sit, pro)] = 30
        if pro == "Heat pump":
            mydata["pro_invcost"][(stf, sit, pro)] = 500000
            mydata["pro_fixcost"][(stf, sit, pro)] = 10000
            mydata["pro_varcost"][(stf, sit, pro)] = 0
            mydata["pro_depreciation"][(stf, sit, pro)] = 15
        if pro == "Electric resistance":
            mydata["pro_invcost"][(stf, sit, pro)] = 150000
            mydata["pro_fixcost"][(stf, sit, pro)] = 5000
            mydata["pro_varcost"][(stf, sit, pro)] = 0
            mydata["pro_depreciation"][(stf, sit, pro)] = 10

    # Dictionaries - conversion ratios
    mydata["ratio_in"] = {
        (2021, "Gas CC", "Gas"): 1.67,
        (2021, "PV plant", "SolarPV"): 1,
        (2021, "Gas heating plant", "Gas"): 1.25,
        (2021, "Solar thermal", "SolarThermal"): 1,
        (2021, "Gas CHP", "Gas"): 2.22,
        (2021, "Heat pump", "Elec"): 1,
        (2021, "Heat pump", "AmbientTemp"): 1,
        (2021, "Electric resistance", "Elec"): 1,
    }
    mydata["ratio_out"] = {
        (2021, "Gas CC", "Elec"): 1,
        (2021, "Gas CC", "CO2"): 0.3,
        (2021, "PV plant", "Elec"): 1,
        (2021, "Gas heating plant", "Heat"): 1,
        (2021, "Gas heating plant", "CO2"): 0.23,
        (2021, "Solar thermal", "Heat"): 1,
        (2021, "Gas CHP", "Elec"): 1,
        (2021, "Gas CHP", "Heat"): 0.88,
        (2021, "Gas CHP", "CO2"): 0.4,
        (2021, "Heat pump", "Heat"): 5,
        (2021, "Electric resistance", "Heat"): 0.95,
    }

    # Dictionaries - time series
    ts_Elec = [30, 30, 35, 30, 35, 60, 100, 120, 80, 60, 55, 50, 50, 60, 65, 60, 85, 100, 130, 140, 120, 100, 65, 50]
    ts_Heat = [30, 30, 35, 30, 35, 60, 100, 120, 80, 60, 55, 50, 50, 60, 65, 60, 85, 100, 130, 140, 120, 100, 65, 50]
    ts_AmbientTemp = [5, 5, 3, 3, 5, 7, 7, 10, 12, 15, 16, 18, 18, 20, 17, 19, 16, 14, 12, 10, 8, 8, 6, 5]
    ts_SolarPV = [0, 0, 0, 0, 0, 0.05, 0.1, 0.15, 0.22, 0.35, 0.4, 0.55, 0.5, 0.45, 0.39, 0.35, 0.3, 0.2, 0.05, 0, 0, 0, 0, 0]
    ts_SolarThermal = [0, 0, 0, 0, 0, 0.05, 0.1, 0.15, 0.22, 0.35, 0.4, 0.55, 0.5, 0.45, 0.39, 0.35, 0.3, 0.2, 0.05, 0, 0, 0, 0, 0]

    # Scale the ambient temperature time series so that the efficiency increases almost linearly with the temperature
    # for the range that we are interested in.
    # Approximation: https://www.researchgate.net/publication/273458507_A_new_two-degree-of-freedom_space_heating_model_for_demand_response/figures?lo=1
    ts_AmbientTemp = np.array(ts_AmbientTemp, dtype=float)
    ts_AmbientTemp = 0.03 * (ts_AmbientTemp + 15) + 3.1
    ts_AmbientTemp[ts_AmbientTemp<=1] = 1
    # What we have here is a COP... let's scale it so that it is lower than 1
    ts_AmbientTemp = ts_AmbientTemp / 5

    mydata["demand"] = {}
    mydata["supim"] = {}
    for stf in mydata["support_timeframes"]:
        for sit in mydata["sites"]:
            mydata["demand"][(stf, sit, "Elec", 0)] = 0
            mydata["demand"][(stf, sit, "Heat", 0)] = 0
            mydata["supim"][(stf, sit, "AmbientTemp", 0)] = 0
            mydata["supim"][(stf, sit, "SolarPV", 0)] = 0
            mydata["supim"][(stf, sit, "SolarThermal", 0)] = 0
            for tm in range(len(ts_Elec)):
                mydata["demand"][(stf, sit, "Elec", tm+1)] = ts_Elec[tm]
                mydata["demand"][(stf, sit, "Heat", tm+1)] = ts_Heat[tm]
                mydata["supim"][(stf, sit, "AmbientTemp", tm+1)] = ts_AmbientTemp[tm]
                mydata["supim"][(stf, sit, "SolarPV", tm+1)] = ts_SolarPV[tm]
                mydata["supim"][(stf, sit, "SolarThermal", tm+1)] = ts_SolarThermal[tm]

    mydata["timesteps"] = range(len(ts_Elec)+1)
    return mydata

### Data visualization

In [ ]:
# Generate the data for the reference scenario
%matplotlib inline
data_ref = generate_scenario()

In [ ]:
# Retrieve the electricity and heat demands
ts_Elec = []
ts_Heat = []
for (stf, sit, com, tm), v in data_ref["demand"].items():
    if ((sit=="OldTown") and (com=="Elec")):
        ts_Elec.append(v)
    if ((sit=="OldTown") and (com=="Heat")):
        ts_Heat.append(v)
# Plot the electricity and heat demands side by side
fig1 = plt.figure(figsize=[12, 5])
ax1a = fig1.add_subplot(1,2,1)
ax1a.set_xlim(1, 24)
ax1a.set_xlabel("tm")
ax1a.set_ylim(0, 150)
ax1a.set_ylabel("SE Elec [MWh]")
ax1a.set_title("Electricity demand")
plt.plot(ts_Elec, color="blue")
fig1.add_subplot(1,2,2)
ax1b = fig1.add_subplot(1,2,2)
ax1b.set_xlim(1, 24)
ax1b.set_xlabel("tm")
ax1b.set_ylim(0, 150)
ax1b.set_ylabel("SE Heat [MWh]")
ax1b.set_title("Heat demand")
plt.plot(ts_Heat, color="orange")

In [ ]:
AmbientTemp = [5, 5, 3, 3, 5, 7, 7, 10, 12, 15, 16, 18, 18, 20, 17, 19, 16, 14, 12, 10, 8, 8, 6, 5]
ts_AmbientTemp = []
for (stf, sit, com, tm), v in data_ref["supim"].items():
    if ((sit=="OldTown") and (com=="AmbientTemp")):
        # COP calculation
        ts_AmbientTemp.append(v * data_ref["ratio_out"][(2021, "Heat pump", "Heat")])
        
# Plot the ambient temperature and the heat pump efficiency in the same plot
fig2, ax2 = plt.subplots(figsize=[10, 5])
#ax2 = plt.axes()
ax2.set_xlim(1, 24)
ax2.set_xlabel("tm")
ax2.set_ylim(0, 22)
ax2.set_ylabel("Ambient temperature [°C]")
ax2.set_title("Temperature")
plt.plot(AmbientTemp, color="red")
ax2b = ax2.twinx()
ax2b.set_ylim(3.5, 4.5)
ax2b.set_ylabel("Heat pump COP [1]")
ax2b.plot(ts_AmbientTemp[1:], color="black")

***
### <span style="color:blue">Task</span>
Plot the capacity factors for SolarPV and SolarThermal side by side.
***

In [ ]:
# Retrieve the SolarPV and SolarThermal timeseries
ts_SolarPV = []
ts_SolarThermal = []
for (stf, sit, com, tm), v in data_ref["supim"].items():
    if ((sit=="OldTown") and (com=="SolarPV")):
        ts_SolarPV.append(v)
    if ((sit=="OldTown") and (com=="SolarThermal")):
        ts_SolarThermal.append(v)
# Plot them side by side
fig1 = plt.figure(figsize=[12, 5])
ax1a = fig1.add_subplot(1,2,1)
ax1a.set_xlim(1, 24)
ax1a.set_xlabel("tm")
ax1a.set_ylim(0, 1)
ax1a.set_ylabel("SE Elec [MWh]")
ax1a.set_title("SolarPV")
plt.plot(ts_SolarPV, color="blue")
fig1.add_subplot(1,2,2)
ax1b = fig1.add_subplot(1,2,2)
ax1b.set_xlim(1, 24)
ax1b.set_xlabel("tm")
ax1b.set_ylim(0, 1)
ax1b.set_ylabel("SE Heat [MWh]")
ax1b.set_title("SolarThermal")
plt.plot(ts_SolarThermal, color="orange")

### Model solving

I moved the script of the model to a separate file to shorten this notebook. It is (almost) identical to the script we used yesterday.

In [ ]:
%run 03_mini_urbs.py

In [ ]:
# Create the model by running the function "create_model"
# This similar to the instantiation of an AbstractModel
model_ref = create_model(data_ref)
# We first load the solver
opt = pyo.SolverFactory('glpk') # glpk: GNU Linear Programming Kit
results = opt.solve(model_ref)
# First way of reporting the solution
results

In [ ]:
results.solver()

### Reporting

In [ ]:
# You can access the variables, for example the output of the processes
supply_data = {}
for (tm, stf, sit, com, com_type), x in model_ref.e_pro_out.items():
    supply_data[(tm, sit, com, com_type)] = pyo.value(x)

df_supply = pd.DataFrame.from_dict(supply_data, orient="index", columns=["SE [MWh] or emissions [t_CO2]"])
df_supply.index = pd.MultiIndex.from_tuples(df_supply.index, names=('t', 'Site', 'Technology', 'Commodity'))
df_supply = df_supply.reorder_levels([1,3,2,0])
df_supply.head()

In [ ]:
df_supply_elec = df_supply[df_supply.index.get_level_values(1) == 'Elec']
df_supply_elec = df_supply_elec.droplevel(1)
df_supply_elec.rename({"SE [MWh] or emissions [kgCO2]": "SE [MWh]"}, axis=1, inplace=True)
df_supply_elec.head()

***
## <span style="color:red">Homework</span>
1. Report the most important results (new capacities, costs, power mix over time) using the techniques that we learned on Day 01.
2. Experiment with different scenarios (i.e. vary the settings when generating the data and solve the different models).
3. Analyze the results and draw some conclusions.
***